In [1]:
!pip install apache-beam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.3/671.3 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 69.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 25.5 MB

In [5]:
pip install apache-beam[interactive]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.0/807.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 71.5 MB/s eta 0:00:00
  Created wheel for timeloop: filename=timeloop-1.0.2-py3-none-any.whl size=3701 sha256=4716424443ee3cc81b5c6c3563b0f8909ab71e5eec20ffcfed9f98d215553d48
  Stored in directory: /root/.cache/pip/wheels/8b/df/32/f72b9fd897c185cd70103331f70e4cb66e3df1de24bd476548
Successfully built timeloop
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgets

In [6]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.window import FixedWindows, TimestampedValue
from datetime import datetime
import random


In [7]:
def assign_random_timestamp(element):
    random_seconds_offset = random.randint(-3600, 3600)  # Offset within the last hour
    random_timestamp = (datetime.now() - datetime(1970, 1, 1)).total_seconds() + random_seconds_offset
    return beam.window.TimestampedValue(element, random_timestamp)


In [8]:
# Define the Beam pipeline
p = beam.Pipeline(options=PipelineOptions())

(p | 'ReadFromCSV' >> beam.io.ReadFromText('/content/users_dataset.csv', skip_header_lines=1)
   | 'ParseCSV' >> beam.Map(lambda line: {'userid': line.split(',')[0], 'name': line.split(',')[1], 'age': line.split(',')[2]})
   | 'AssignRandomTimestamps' >> beam.Map(assign_random_timestamp)
   | 'WindowInto' >> beam.WindowInto(FixedWindows(3600))
   | 'GroupByAge' >> beam.Map(lambda x: (x['age'], 1))
   | 'CountByAge' >> beam.CombinePerKey(sum)
   | 'WriteResults' >> beam.io.WriteToText('output_windowed_results')
)

# Run the pipeline
result = p.run()
result.wait_until_finish()


'DONE'